<a href="https://colab.research.google.com/github/dicksontsai/intro_to_python_colabs/blob/main/Intro_to_Python_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Python with Google Colab (Part 2)

by Dickson Tsai

The target audience is high schoolers completely new to programming but have gone through [Part 1](https://colab.research.google.com/drive/1cufid_EX4qzz9tKMQqGcxX2mJ_42PmFo#scrollTo=8PB07imLL9gz).

In this tutorial, we will cover:
* List comprehensions
* Dictionaries
  - Example application: JSON
* Classes
  - Example application: Trees
* While loops
* Higher-order functions (functions that take in/return functions)
* Recursion
  - Example application: Your first programming interview question!

We will not cover anything new from numpy or Matplotlib. They will be in separate parts.

## List Comprehensions

Let's review some concepts we learned in the last part. Here, we make a list that contains the square of the numbers in `nums`.

In [ ]:
nums = [0, 1, 2, 3, 4]
squares = []
for x in nums:
    squares.append(x ** 2)
print(squares)

When programming, we often want to convert one list into another. Python provides the **list comprehension** pattern to simplify this:

```
[<expr> for <elem> in <original_list>]
```

For example, we can write the above as:

In [ ]:
squares_2 = [x ** 2 for x in nums]
print(squares, squares2)

You can also decide which elements not to include in your list comprehension:

```
[<expr> for <elem> in <original_list> if <cond>]
```

In [ ]:
even_squares = [x ** 2 for x in nums if x % 2 == 0]
print(even_squares)

## Dictionaries

A **dictionary** stores (key, value) pairs. You can use it like this:

d = {'cat': 'cute', 'dog': 'furry'}

To get a value from your dictionary, use the same `[]` from lists, but put in your key this time.

In [ ]:
print(d['cat'])
print('cat' in d)

In [ ]:
d['fish'] = 'wet'    # Set an entry in a dictionary
print(d['fish'])      # Prints "wet"

In [ ]:
print(d['monkey'])  # KeyError: 'monkey' not a key of d

In [ ]:
print(d.get('monkey', 'N/A'))  # Get an element with a default; prints "N/A"
print(d.get('fish', 'N/A'))    # Get an element with a default; prints "wet"

In [ ]:
del d['fish']        # Remove an element from a dictionary
print(d.get('fish', 'N/A')) # "fish" is no longer a key; prints "N/A"

You can find all you need to know about dictionaries in the [documentation](https://docs.python.org/3/library/stdtypes.html#dict).

It is easy to iterate over the `.keys()`, `.values()`, or `.items()` of a dictionary. Note that dictionaries do NOT preserve the order of your items.

In [ ]:
d = {'person': 2, 'cat': 4, 'spider': 8}
for animal, legs in d.items():
    print('A {} has {} legs'.format(animal, legs))

Dictionary comprehensions: These are similar to list comprehensions, but allow you to easily construct dictionaries. For example:

In [ ]:
nums = [0, 1, 2, 3, 4]
even_num_to_square = {x: x ** 2 for x in nums if x % 2 == 0}
print(even_num_to_square)

### Example Application: JSON

JSON (JavaScript Object Notation) is a simple standard for exchanging data. Many institutions such as the World Bank offer data for free\* (free when your usage is reasonable) in JSON format. Here, we will explore the US population from 1960 to 2019.

First, we will use the requests library to make a request to World Bank's web
server for the data.

In [ ]:
# Make sure to run this only once.
import requests

response = requests.get("http://api.worldbank.org/v2/countries/USA/indicators/SP.POP.TOTL?per_page=5000&format=json")

Then, let's examine the JSON data.

In [ ]:
response.json()

It seems like the data is a list where the first element is just metadata about the dataset and the second element is another list with the actual population data. Let's extract that part.

In [ ]:
pop_data = response.json()[1]

Then, we simply care about the years (from the 'date' field) and the numbers (from the 'value' field). Try doing this yourself, then see my solution below.

In [ ]:
years = [record['date'] for record in pop_data]
pop = [record['value'] for record in pop_data]
print(years[:5], pop[:5])

Now, let's plot this data!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Recall that Numpy arrays are easier to work with than Python lists for data.
years_arr = np.array(years)
pop_arr = np.array(pop)
# Years is not in sorted order. We can find the indexes that sorts the years.
sorted_year_idxs = np.argsort(years_arr)
# Then sort both arrays by the indexes.
plt.plot(years_arr[sorted_year_idxs], pop_arr[sorted_year_idxs])
plt.xlabel('Year')
# Only show ticks every 5 years.
plt.xticks(np.arange(0, len(pop_arr), 5))
plt.ylabel('Population')
plt.title('US Population')

For more JSON datasets, check out Justin Dorfman's [curated list on Github](https://github.com/jdorfman/awesome-json-datasets).

## Classes

So far, we have seen many types of data:
* int
* string
* bool
* list
* dict
* ...

Everything in Python is an object of a particular type. You can use the built-in `type()` function to determine the type of the object.

Objects not only *store* some data but also have *special* code for handling its data based on their type. This "special code" belonging to the type are known as **methods**.

type | data stored | methods
-----|-------------|--------------------
int  | an integer  |
string| characters | .upper(), .lower(), ...
list | elements | .append(), .pop(), ...


What if you wanted to define your own types? This is where the `class` keyword comes in.

A **class** is a blueprint for producing a certain type of object. In fact, classes are often used to represent the type of the object they produce. For example, an object produced by `class Car` has type `Car`.

Below is an example class definition. A class definition consists of:

1. The `class` keyword.
1. The name of the class.
1. An `__init__` function to "initialize" a new instance. This means populating the minimum data necessary for the object to make sense. For example, a `Car` object needs to know its make and model.
  - `self` is a reference to the new instance you're creating.
1. Methods. Each method takes in `self` as a parameter, which refers to the instance that the method is called on.

In [ ]:
class Car:
  def __init__(self, make, model):
    self.make = make
    self.model = model
    self.gas_tank = 0
  
  def fill_gas(self):
    # Assume the gas tank can store 20 gallons.
    self.gas_tank = 20

  def drive(self, miles):
    # Assume the car drives 30 miles per gallon.
    self.gas_tank = self.gas_tank - miles / 30

To create an **instance** of a `Car` from our blueprint, call `Car()` with the required arguments. Each call to `Car()` creates a different object, each managing its own data.

In [ ]:
my_car = Car("Toyota", "Camry")
your_car = Car("Ford", "F150")
my_car.fill_gas()
print(my_car.gas_tank, your_car.gas_tank)
print(my_car.make, your_car.make)

Classes are so useful as an **organizational** tool for your programs. If you are very disciplined about the types you work with, you will find code easy to write and organize.

### Exercise: Credit Card Class

In this exercise, we will design a class to model a credit card.

Here is what your `CreditCard` class should support:

method   |    description
---------|-------------------
`__init__(self, interest_rate, credit_limit)` | You're provided an `interest_rate` and `credit_limit`
`charge(self, purchase_cost)` | Add a charge to the card. Return true if successful, or false if the charge will exceed the user's credit limit.
`pay(self, payment)` | Accept a payment from the user. If the payment exceeds the current balance, return the excess money to the user. (How would you handle negative input?)


#### Solution

While not explicitly specified in the problem statement, you'll need to keep track of the credit card balance yourself.

In [ ]:
class CreditCard:
  def __init__(self, interest_rate, credit_limit):
    self.interest_rate = interest_rate
    self.credit_limit = credit_limit
    self.balance = 0

  def charge(self, purchase_cost):
    if purchase_cost + self.balance > self.credit_limit:
      return False
    self.balance += purchase_cost
    return True

  def pay(self, payment):
    if payment <= 0:
      return 0
    actual_payment = min(self.balance, payment)
    self.balance = self.balance - actual_payment
    return payment - actual_payment

cc = CreditCard(0.23, 5000)
print(cc.charge(500))
print(cc.charge(10000))
print(cc.balance)
print(cc.pay(20))
print(cc.balance)
print(cc.pay(700))
print(cc.balance)

## While Loops

Use the `while` keyword to execute a piece of code over and over again **while** a certain condition is true:
```
while <cond is true>:
    <execute this code>
```

In [ ]:
i = 0
while i < 5:
  print(i)
  i += 1

You may be wondering why it has taken this long to cover `while` loops. It seems so basic! Well, it is also extremely **error-prone**. Remember to update your variable in the condition. Otherwise, your program will enter an "infinite loop", where it never stops until the operating system steps in and stops your program.

## Recursion

Recursion is where you call the same function within itself.

Of course, your recursive code must stop at some point. Here are the key ingredients for your recursive function:

1. Base case(s): Cases so trivial that you just return a specific value.
1. Recursive call(s): Calls to the same function. Generally, your recursive calls should "work towards" your base cases.

Here's a classic example: The factorial function.

In [ ]:
def factorial(x):
  if x == 0:
    return 1
  return x * factorial(x-1)

print(factorial(4))

* Base case: If x = 0, then by definition, 0! is 1.
* Recursive call: 4! = 4 \* 3 \* 2 \* 1. Notice how the second part is simply 3!. Thus, you can say 4! = 4 * 3!.
  - 3! is "working towards" the base case of 0!.

### Example Application: Your First Programming Interview Question

Companies use programming interview questions to assess a candidate's programming ability. [Leetcode](https://leetcode.com) is a website that hosts a rich set of programming interview questions.

To give you a taste of what these questions are like, let's go through the problem ["Number of Islands"](https://leetcode.com/problems/number-of-islands/) together. 

Example 1:
```
Input: grid = [
  ["1","1","1","1","0"],
  ["1","1","0","1","0"],
  ["1","1","0","0","0"],
  ["0","0","0","0","0"]
]
Output: 1
```
Example 2:
```
Input: grid = [
  ["1","1","0","0","0"],
  ["1","1","0","0","0"],
  ["0","0","1","0","0"],
  ["0","0","0","1","1"]
]
Output: 3
```

#### Solution

The basic idea is to count each island once and ensure that you don't count the same island again. This involves keeping track of which coordinates you have already visited. When you land on an island, you will explore the entire island and mark all of its coordinates as visited. This exploration is naturally expressed as a recursive call -- just make sure your base cases cover all bases.

In [ ]:
def number_of_islands(grid):
  visited = [[False for _ in row] for row in grid]
  x_len = len(grid)
  if x_len == 0:
    return 0
  y_len = len(grid[0])

  # This solution has one small inefficiency. What's one way of speeding it up?
  # Hint: Consider what the stack will look like when this code is run.
  def explore(x, y):
    # Base case: Out of bounds
    if x < 0 or x >= x_len:
      return
    if y < 0 or y >= y_len:
      return 
    # Base case: Already visited
    if visited[x][y]:
      return
    visited[x][y] = True
    if grid[x][y] == "1":
      explore(x-1, y)
      explore(x, y-1)
      explore(x+1, y)
      explore(x, y+1)

  num_islands = 0
  for x in range(x_len):
    for y in range(y_len):
      if grid[x][y] == "1" and not visited[x][y]:
        num_islands += 1
        explore(x, y)
  
  return num_islands

print(number_of_islands(
  [
    ["1","1","1","1","0"],
    ["1","1","0","1","0"],
    ["1","1","0","0","0"],
    ["0","0","0","0","0"]
  ]
))
print(number_of_islands(
  [
    ["1","1","0","0","0"],
    ["1","1","0","0","0"],
    ["0","0","1","0","0"],
    ["0","0","0","1","1"]
  ]
))

### Example Application: Trees

Trees are a widely used **abstract data type**, a data type that is a general pattern of many possible data types, that represents hierarchical information.

Below is an example of a simple tree class.

In [ ]:
class Tree:
  def __init__(self, val, children=[]):
    self.val = val
    self.children = children

  def __str__(self):
    return "({} [{}])".format(self.val, " ".join([str(child) for child in self.children]))

In [ ]:
a = Tree(1, [Tree(2, [Tree(3), Tree(4)]), Tree(5)])
print(a)

## Higher Order Functions

Functions can be treated a values themselves. That means functions can be the arguments or return values of other functions!

### Functions as input

In [ ]:
def filter_list(lst, filter_fn):
  return [elem for elem in lst if filter_fn(elem)]

def is_even(x):
  return x % 2 == 0

def is_large(x):
  return x > 5

print(filter_list([1, 2, 3, 4, 5], is_even))
print(filter_list([1, 100, 2, 200], is_large))

### Functions defined in other functions

In [ ]:
def lookup_func_for_table(table):
  # This inner function has access to everything from the outer function.
  def lookup(key):
    return table(key)
  return lookup

lookup_for_age = lookup_func_for_table({
    'Alex': 10,
    'Brianne': 5,
    'Cameron': 8,
})
print(lookup_for_age('Alex'))

## Conclusion

Congratulations on making it this far! At this point, you should have a lot of the basics under your belt, though there's always more to learn!

Next steps:

* Learn how to run Python on your own computer.
  - Learn how to edit files using an IDE (e.g. VSCode) or text editor (e.g. Vim).
* Discover project ideas online. Online courses and college courses are a great place to start.
* Build your own project and scripts.
  - One script I use is to print out every day of the month in a format like `(Mon) 2021/03/21` to paste in my Google Docs diary.
* Learn about algorithms and runtime complexity to write more efficient code.
  - Tackle more programming interview questions.